# Learning rate scheduling

In [1]:
from twilio.rest import Client

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.datasets import make_classification

import tensorflow as tf
from tensorflow import keras 

import numpy as np

D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
def send_message(text):
    account_sid = 'AC627ada27fffdce01d175164bcba0d23c' #Can be obtained from Twilio Console
    auth_token = '729c156bf50bc7bc1fb49c2256cd9836'  #Can be obtained from Twilio Console
    client = Client(account_sid, auth_token)

    message = client.messages \
    .create(
         from_='whatsapp:+14155238886',
         body=text,
         to='whatsapp:+917400452552'
     )

In [7]:
class CustomLearningRateScheduler(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        send_message("\nEpoch %03d: Learning rate is %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05),
    (6, 0.01),
    (9, 0.005),
    (12, 0.001),
]


def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr

In [8]:
cb = CustomLearningRateScheduler(lr_schedule) #Creating the Callback

In [9]:
x,y = make_classification(n_samples=1000, n_classes=2, n_features=20)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

In [10]:
def create_model():
    model = Sequential()
    model.add(Dense(60, input_dim=20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.RMSprop(learning_rate=0.1), metrics=["mean_absolute_error"])

    return model

model = create_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model.fit(x_train,y_train,batch_size=32, epochs=20, callbacks=[cb], verbose=1) #Fitting the model

Epoch 1/20
700/700 [==============================] - 0s 178us/sample - loss: 0.1719 - mean_absolute_error: 0.2000
Epoch 2/20
700/700 [==============================] - 0s 83us/sample - loss: 0.1224 - mean_absolute_error: 0.1288
Epoch 3/20
700/700 [==============================] - 0s 94us/sample - loss: 0.1200 - mean_absolute_error: 0.1252
Epoch 4/20
700/700 [==============================] - 0s 100us/sample - loss: 0.1025 - mean_absolute_error: 0.1071
Epoch 5/20
700/700 [==============================] - 0s 80us/sample - loss: 0.0807 - mean_absolute_error: 0.0844
Epoch 6/20
700/700 [==============================] - 0s 88us/sample - loss: 0.0831 - mean_absolute_error: 0.0881
Epoch 7/20
700/700 [==============================] - 0s 74us/sample - loss: 0.0766 - mean_absolute_error: 0.0777
Epoch 8/20
700/700 [==============================] - 0s 73us/sample - loss: 0.0752 - mean_absolute_error: 0.0764
Epoch 9/20
700/700 [==============================] - 0s 33us/sample - loss: 0.0679 - 